In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.Compose([
    transforms.ToTensor(),               
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

dataset = datasets.ImageFolder(root='animals_resized', transform=transform)
batch_size = 30
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
num_epochs = 200
z = 100


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 512 * 8 * 8),
            nn.ReLU(),
            nn.BatchNorm1d(512 * 8 * 8),
            nn.Unflatten(1, (512, 8, 8)),

            nn.ConvTranspose2d(512,256,4,2,1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            
            nn.ConvTranspose2d(256,128,4,2,1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            
            nn.ConvTranspose2d(128,64,4,2,1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            
            nn.ConvTranspose2d(64,3,4,2,1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)
    

class Discriminator(nn.Module):
    def __init__(self,):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 16,4,2,1), 
            nn.LeakyReLU(0.2),  
            
            nn.Conv2d(16,32,4,2,1), 
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(32,64,4,2,1), 
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(64,128,4,2,1),  
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            
            nn.Flatten(),  
            nn.Linear(128 * 8 * 8, 1), 
            nn.Sigmoid()  
        )
    
    def forward(self, y):
        return self.model(y)

generator = Generator().to(device)
discriminator = Discriminator().to(device)

criterion = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

G_losses = []
D_losses = []

for epoch in range(num_epochs):
    G_loss_epoch = 0.0
    D_loss_epoch = 0.0
    for batch in dataloader:
        real_imag = batch[0].to(device)
        real_labels = torch.ones(batch_size, 1).to(device)  
        fake_labels = torch.zeros(batch_size, 1).to(device)
        optimizer_d.zero_grad()
        real_op = discriminator(real_imag)
        real_loss = criterion(real_op, real_labels)
        x= torch.randn(batch_size,z).to(device)
        fake_imag = generator(x)
        fake_op = discriminator(fake_imag.detach())
        fake_loss = criterion(fake_op,fake_labels)
        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_d.step()

        optimizer_g.zero_grad()
        fake_op_g = discriminator(fake_imag)
        g_loss = criterion(fake_op_g,real_labels)
        g_loss.backward()
        optimizer_g.step()

        G_loss_epoch += g_loss.item()
        D_loss_epoch += d_loss.item()

    avg_G_loss = G_loss_epoch / len(dataloader)
    avg_D_loss = D_loss_epoch / len(dataloader)

    G_losses.append(avg_G_loss)
    D_losses.append(avg_D_loss)
    print(f"Epoch {epoch+1}/{num_epochs} | D Loss: {avg_D_loss:.4f} | G Loss: {avg_G_loss:.4f}")

import torch
import matplotlib.pyplot as plt
import numpy as np


generator.eval()
z = torch.randn(1, 100).to(device)  

with torch.no_grad():  
    generated_image = generator(z).cpu()


def denormalize(image_tensor):
    image_tensor = (image_tensor * 0.5) + 0.5  
    return image_tensor.numpy()

generated_image = denormalize(generated_image.squeeze())

import matplotlib.pyplot as plt
plt.imshow(np.transpose(generated_image, (1, 2, 0)))  
plt.axis('off') 
plt.show()

        
